In [70]:
connect-mggraph -scopes "UserAuthenticationMethod.Read.All, User.Read.All"

Welcome to Microsoft Graph!

Connected via delegated access using 14d82eec-204b-4c2f-b7e8-296a70dab67e
Readme: https://aka.ms/graph/sdk/powershell
SDK Docs: https://aka.ms/graph/sdk/powershell/docs
API Docs: https://aka.ms/graph/docs

NOTE: You can use the -NoWelcome parameter to suppress this message.



In [71]:
$u = get-mguser

In [72]:
$u


DisplayName       Id                                   Mail                              UserPrinci
                                                                                         palName
-----------       --                                   ----                              ----------
Adele Vance       e065d899-0464-4a43-b02a-206028b27982 AdeleV@61b3yr.onmicrosoft.com     AdeleV@61…
Alex Wilber       c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726 AlexW@61b3yr.onmicrosoft.com      AlexW@61b…
Daniel Carmack    9a2e7c71-2c59-4b80-ada4-19d8534e6d99 Daniel.C@61b3yr.onmicrosoft.com   Daniel.C@…
Diego Siciliani   0a02a0f1-48e1-4010-b904-c7650b34c112 DiegoS@61b3yr.onmicrosoft.com     DiegoS@61…
Grady Archie      2ea73151-a011-47b5-9a99-8d37adc221b5 GradyA@61b3yr.onmicrosoft.com     GradyA@61…
Henrietta Mueller 2c758750-2644-4e13-975f-453e0dfa2263 HenriettaM@61b3yr.onmicrosoft.com Henrietta…
Isaiah Langer     f243a1ce-a5e6-42e1-abe1-f49ca450d6a0 IsaiahL@61b3yr.onmicrosoft.com    IsaiahL@6…
Jo

In [74]:
$Id9 = $u[9].Id


In [75]:
$Id9

98d68316-30d8-43c4-ac35-4319ec3b50e9


In [80]:
get-mguserAuthenticationmethod -userid $id9 -All



Id
--
3179e48a-750b-4051-897c-87b9720928f7
28c10230-6103-485e-b985-444c60001490
enBnDvzucU792UQaLOlwywon5sdtKb0EBTVq-JJFZkk1
Pesj_F4ljvb_GUcBPQTOFVmE2qaqwb-sI004i-d1VAE1
tT97bttgYTvaiXpKK9jrSPb2OBrR1QaK6zI3M56INFw1
6q91Mrs-H44VzaRduejpkyJOSak0_8a1TYvuLbc5n1U1
6f17213b-b1ad-48df-b682-ecd4f437a2ea



In [77]:
$method | get-member


   TypeName: Microsoft.Graph.PowerShell.Models.MicrosoftGraphAuthenticationMethod

Name                 MemberType            Definition
----                 ----------            ----------
Add                  Method                void Add(string key, System.Object value), void IAssoci…
Clear                Method                void Clear(), void IAssociativeArray[Object].Clear()
ContainsKey          Method                bool ContainsKey(string key), bool IAssociativeArray[Ob…
CopyFrom             Method                void CopyFrom(System.Collections.IDictionary source), v…
Equals               Method                bool Equals(System.Object obj)
GetHashCode          Method                int GetHashCode()
GetType              Method                type GetType()
Remove               Method                bool Remove(string key), bool IAssociativeArray[Object]…
ToJson               Method                Microsoft.Graph.PowerShell.Runtime.Json.JsonNode, Micro…
ToJsonString       

In [78]:
$method | ForEach-Object -Process {$_.ToJsonString()} | Format-List

{
  "@odata.type": "#microsoft.graph.phoneAuthenticationMethod",
  "phoneNumber": "+852 25028662",
  "phoneType": "mobile",
  "smsSignInState": "notAllowedByPolicy",
  "id": "3179e48a-750b-4051-897c-87b9720928f7"
}
{
  "@odata.type": "#microsoft.graph.passwordAuthenticationMethod",
  "createdDateTime": "2023-12-27T03:40:17Z",
  "id": "28c10230-6103-485e-b985-444c60001490"
}
{
  "@odata.type": "#microsoft.graph.windowsHelloForBusinessAuthenticationMethod",
  "displayName": "DESKTOP-G9TFT8N",
  "createdDateTime": "2023-12-30T10:57:23Z",
  "keyStrength": "normal",
  "id": "enBnDvzucU792UQaLOlwywon5sdtKb0EBTVq-JJFZkk1"
}
{
  "@odata.type": "#microsoft.graph.windowsHelloForBusinessAuthenticationMethod",
  "displayName": "",
  "createdDateTime": "2023-12-30T00:03:14Z",
  "keyStrength": "normal",
  "id": "Pesj_F4ljvb_GUcBPQTOFVmE2qaqwb-sI004i-d1VAE1"
}
{
  "@odata.type": "#microsoft.graph.windowsHelloForBusinessAuthenticationMethod",
  "displayName": "LC21194",
  "createdDateTime": "2023-12-2

In [84]:
$u | ForEach-Object -Process {
                           $methods = get-mguserAuthenticationmethod -userid $_.Id
                           $hasMFASetup = $false
                           foreach ($m in $methods) {
                              $extraProps = $m.ToJsonString()
                              $_.DisplayName
                              $extraProps | Format-Table @{Label = "DisplayName" ; Expression={$_.DisplayName}}
                           } 
                     }

Adele Vance
{
  "@odata.type": "#microsoft.graph.passwordAuthenticationMethod",
  "createdDateTime": "2023-12-28T03:32:47Z",
  "id": "28c10230-6103-485e-b985-444c60001490"
}
Alex Wilber
{
  "@odata.type": "#microsoft.graph.passwordAuthenticationMethod",
  "createdDateTime": "2023-12-26T10:12:23Z",
  "id": "28c10230-6103-485e-b985-444c60001490"
}
Daniel Carmack
{
  "@odata.type": "#microsoft.graph.passwordAuthenticationMethod",
  "createdDateTime": "2024-02-06T07:19:03Z",
  "id": "28c10230-6103-485e-b985-444c60001490"
}
Diego Siciliani
{
  "@odata.type": "#microsoft.graph.passwordAuthenticationMethod",
  "createdDateTime": "2023-12-26T10:12:23Z",
  "id": "28c10230-6103-485e-b985-444c60001490"
}
Grady Archie
{
  "@odata.type": "#microsoft.graph.passwordAuthenticationMethod",
  "createdDateTime": "2023-12-26T10:12:24Z",
  "id": "28c10230-6103-485e-b985-444c60001490"
}
Henrietta Mueller
{
  "@odata.type": "#microsoft.graph.passwordAuthenticationMethod",
  "createdDateTime": "2023-12-26T10: